### Разбивка снимков на более крупные сектора для скачивания синоптических данных

In [ ]:
import pandas as pd

# Чтение данных
ds = pd.read_csv('dataset_veg_index.csv', index_col=0)

def calculate_sector_dimensions(ds):
    """Создает словарь с размерами (x, y) для каждого уникального sector_id."""
    sector_dict = {}
    for id in ds.sector_id.unique():
        x = ds[ds.sector_id == id].lon.nunique()
        y = ds[ds.sector_id == id].lat.nunique()
        sector_dict[id] = (x, y)
    return sector_dict

def get_macro_px_id(M, N, sector_x, sector_y, cell_size):
    """Вычисляет macro_px_id на основе размеров и положения."""
    x_cells = (M + cell_size - 1) // cell_size
    y_cells = (N + cell_size - 1) // cell_size
    
    cell_id = sector_x // cell_size + (sector_y // cell_size) * x_cells
    return cell_id

def get_sector_idx(ds):
    """Добавляет в DataFrame столбец sector_idx."""
    ds['sector_idx'] = 0
    set_unique = set()
    counter = 0
    for idx, row in ds.iterrows():
        uniq = f"{row.date}_{row.sector_id}"
        if uniq not in set_unique:
            counter = 0
            set_unique.add(uniq)
        ds.loc[idx, 'sector_idx'] = counter
        counter += 1
    return ds

def get_sector_xy(idx, M, N):
    """Возвращает координаты сектора (x, y) на основе индекса."""
    sector_x = idx % M
    sector_y = idx // M
    return sector_x, sector_y

def is_center(sector_x, sector_y, cell_size):
    """Создает булевый флаг, указывающий, является ли сектор центральным."""
    return (sector_x % cell_size == cell_size // 2) and (sector_y % cell_size == cell_size // 2)

def process_dataset(ds, sector_dict, cell_size=19): # cell_size - размер новой ячейки
    """Обрабатывает DataFrame, добавляя столбцы 'macro_px_id' и 'is_center'."""
    ds['macro_px_id'] = 0
    ds['is_center'] = 0
    
    for idx, row in ds.iterrows():
        sector_id = row.sector_id
        M, N = sector_dict[sector_id]
        sector_x, sector_y = get_sector_xy(row.sector_idx, M, N)
        ds.loc[idx, 'macro_px_id'] = get_macro_px_id(M, N, sector_x, sector_y, cell_size)
        ds.loc[idx, 'is_center'] = int(is_center(sector_x, sector_y, cell_size))
    
    return ds

# Основной блок выполнения
sector_dict = calculate_sector_dimensions(ds)
ds = get_sector_idx(ds)
ds = process_dataset(ds, sector_dict)

# Сохранение результата
ds.to_csv('dataset_veg_index_cell.csv', index=False)

### Получение данных о погоде с помощью API OpenWeatherMap

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
import time
from tqdm import tqdm


def get_wether_by_api(df):
    # df columns: date, lat, lon!!!
    
    # Создаем список с начальными датами, которые на неделю раньше
    start_dates = pd.to_datetime(df['date']) - pd.Timedelta(weeks=1)

    # Преобразуем начальные даты в строки в формате 'YYYY-MM-DD'
    start_date_list = start_dates.dt.strftime('%Y-%m-%d').tolist()

    # Преобразуем остальные столбцы в списки
    lat_list = df['lat'].tolist()
    lon_list = df['lon'].tolist()
    end_date_list = df['date'].tolist()  
    sector_id = df['sector_id'].tolist()
    macro_px_id = df['macro_px_id'].tolist()
    
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Инициализируем пустые списки для хранения промежуточных DataFrame
    final_datasets = []
    
    pbar = tqdm(total=len(lat_list)//80, desc="Processing data", unit="locations", )
    
    for i in range(0, len(lat_list), 80):
        final_dataframes = []
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat_list[i:i+80],
            "longitude": lon_list[i:i+80],
            "start_date": start_date_list[i:i+80],
            "end_date": end_date_list[i:i+80],
            "hourly": "relative_humidity_2m",
            "daily": ["temperature_2m_mean", "rain_sum", "wind_speed_10m_max"],
            "timezone": "auto"
        }
        
        batch_sector_id = sector_id[i:i+80]
        batch_macro_px_id = macro_px_id[i:i+80]
        responses = openmeteo.weather_api(url, params=params)
        
        # Итерация по всем ответам в списке responses
        assert len(responses) == len(batch_sector_id) == len(batch_macro_px_id)
        for response, sector, px in zip(responses, batch_sector_id, batch_macro_px_id):
            
            # Извлекаем координаты
            latitude = response.Latitude()
            longitude = response.Longitude()
            
            # Получаем и обрабатываем почасовые данные
            hourly = response.Hourly()
            hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()

            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="right"
                ),
                "relative_humidity_2m": hourly_relative_humidity_2m
            }
            
            hourly_dataframe = pd.DataFrame(data=hourly_data)
            hourly_dataframe["lat"] = latitude
            hourly_dataframe["lon"] = longitude
            
            
            # Группируем по дате и рассчитываем среднее значение влажности за день
            hourly_dataframe = hourly_dataframe.groupby(["date", "lat", "lon"]).agg({
                "relative_humidity_2m": "mean"
            }).reset_index()
            
            # Получаем и обрабатываем ежедневные данные
            daily = response.Daily()
            daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
            daily_rain_sum = daily.Variables(1).ValuesAsNumpy()
            daily_wind_speed_10m_max = daily.Variables(2).ValuesAsNumpy()

            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="right"
                ),
                "temperature_2m_mean": daily_temperature_2m_mean,
                "rain_sum": daily_rain_sum,
                "wind_speed_10m_max": daily_wind_speed_10m_max
            }
            
            daily_dataframe = pd.DataFrame(data=daily_data)
            daily_dataframe["lat"] = latitude
            daily_dataframe["lon"] = longitude
            daily_dataframe["sector_id"] = sector
            daily_dataframe["macro_px_id"] = px
            
            # Объединяем почасовые и ежедневные данные по дате и координатам
            merged_dataframe = pd.merge(hourly_dataframe, daily_dataframe, on=["date", "lat", "lon"], how="inner")
            
            # Добавляем объединенный DataFrame в список финальных DataFrame
            final_dataframes.append(merged_dataframe)

        # Объединяем все DataFrame из списка в один финальный DataFrame
        final_dataframe = pd.concat(final_dataframes, ignore_index=True)
        
        final_datasets.append(final_dataframe)
        
        # Задержка в 6 секунды между запросами к API для предотвращения блокировки IP-адреса сервером Open-Meteo API 
        time.sleep(10)
        pbar.update(1)
        
    # Объединяем все финальные DataFrame в один общий DataFrame
    final_dataset = pd.concat(final_datasets, ignore_index=True)
    
    return final_dataset

# Читаем данные из файла
df = pd.read_csv('dataset_veg_index_cell.csv')

# Оставляем только центральные ячейки
df_central = df[df.is_center == 1].reset_index(drop=True)

# Вызываем функцию get_wether_by_api и сохраняем результат в переменную res
res = get_wether_by_api(df_central)

# Обрабатываем результат
res.date = res.date.dt.strftime('%Y-%m-%d')
res.drop(columns=['lat', 'lon'], inplace=True)
res.to_csv('weather.csv', index=False)

# Объединяем исходный DataFrame с полученными данными о погоде
final_df = pd.merge(df, res, on=['sector_id', 'macro_px_id'], how='left', suffixes=('_init', '_weather'))

# Обработка датасета
final_df.rename(columns={'date_init': 'date'}, inplace=True)
final_df.drop(columns=['sector_idx', 'macro_px_id', 'is_center'], inplace=True)
final_df.dropna(inplace=True)

# Запиём полученный DataFrame в файл
final_df.to_csv('dataset_veg_index_cell_weather.csv', index=False)
